### 하나은행 예금 데이터 수집
- requests, bs4 패키지를 사용하여 하나은행 예금 데이터를 아래와 같이 수집할 예정
    - 상품에 대한 금리는 최고 금리가 출력되도록!
    - items_df : 상품 리스트에 대한 데이터 프레임
    - items_detail_df : 상품에 대한 상세 정보를 가지는 데이터 프레임
- URL : https://www.kebhana.com/cont/mall/mall08/mall0805/index.jsp?_menuNo=62608

In [6]:
import requests
from bs4 import BeautifulSoup

In [7]:
# 예금 데이터 수집 함수
def get_list():
    url = "https://www.kebhana.com/cont/mall/mall08/mall0805/index.jsp?_menuNo=62608"
    response = requests.get(url)
    dom = BeautifulSoup(response.content, "html.parser")
    elements = dom.select(".product-list > li")
    datas=[]
    
    for element in elements:
        datas.append({
        "product-title" : element.select_one(".product-tit > i").text,
        "title" : element.select_one("em > a").text,
        "detail" : element.select_one(".tit-desc > a").text,
        "link" : "https://www.kebhana.com" + element.select_one("em > a").get("href")
        })
        
    return pd.DataFrame(datas)

In [8]:
items_df = get_list()
items_df

,product-title,title,detail,link
0,정기예금,하나 원큐 정기예금,간편하게 우대금리를 제공하는 스마트폰 예금,https://www.kebhana.com/cont/mall/mall08/mall0...
1,정기예금,하나머니세상 정기예금,하나멤버스 회원손님이 예금의 이자를 하나머니로 적립동의하면 우대금리 제공! 하나카드...,https://www.kebhana.com/cont/mall/mall08/mall0...
2,정기예금,N플러스 정기예금,언제 어디서나 편리하게 스마트폰으로 정기예금을 가입하세요~,https://www.kebhana.com/cont/mall/mall08/mall0...
3,정기예금,e-플러스 정기예금,적은 금액이라도 높은금리 혜택을 마음껏 누리세요,https://www.kebhana.com/cont/mall/mall08/mall0...
4,정기예금,리틀빅 정기예금,오늘도 열심히 살아가는 이 시대의 작은 영웅 당신을『리틀빅 정기예금』이 응원합니다!...,https://www.kebhana.com/cont/mall/mall08/mall0...
5,정기예금,주거래 정기예금,입출금 거래가 늘어날수록 정기예금 금리도 쑥~쑥!KEB하나은행 주거래 고객을 위한 ...,https://www.kebhana.com/cont/mall/mall08/mall0...
6,정기예금,행복knowhow 연금예금,"노후자금, 생활자금, 재투자자금까지! 행복knowhow 연금예금으로 설계하세요!",https://www.kebhana.com/cont/mall/mall08/mall0...
7,정기예금,3·6·9 정기예금,"3개월마다, 기쁜 날마다, 고금리의 즐거움을 드립니다.3개월마다, 기쁜 날마다 언제...",https://www.kebhana.com/cont/mall/mall08/mall0...
8,정기예금,고단위 플러스(금리연동형),이자 지급 방법도 내 맘대로! 이자 지급 시기도 내 맘대로!,https://www.kebhana.com/cont/mall/mall08/mall0...
9,정기예금,고단위 플러스(금리확정형),이자 지급 방법도 내 맘대로! 이자 지급 시기도 내 맘대로!,https://www.kebhana.com/cont/mall/mall08/mall0...


In [ ]:
from selenium import webdriver
import time

In [9]:
# 예금 데이터 상세페이지 수집 함수
info = []

def detail_datas(d_link, idx):
    url = d_link
    response = requests.get(url)
    dom = BeautifulSoup(response.content, "html.parser")
    element = dom.select(".prodcutInfo")[0]

    for i in range(1,len(element.select("dt"))+1) :
        
        if element.select_one("dt:nth-of-type({})".format(i)).text != "금리" :
            info.append({
            "product_idx" : idx,
            "title_txt" : element.select_one("dt:nth-of-type({})".format(i)).text,
            "desc_txt" : element.select_one("dd:nth-of-type({})".format(i)).text\
                .replace("\n","").replace("\r","").replace("\t","")
            })
        else :  
            i_url = "https://www.kebhana.com" + element.select_one("#zff").get("src")
            response = requests.get(i_url)
            desc_txt = BeautifulSoup(response.text, "html.parser").text.strip().replace("\n", "")
            numbers = [float(data) for data in re.findall("([0-9]{1,2}\.?[0-9]{0,2})%", desc_txt)]
            desc_txt = np.max(numbers)

    return info

In [261]:
# 예금 데이터 상세페이지 수집 함수
info = []

def detail_datas(d_link, idx):
    url = d_link
    response = requests.get(url)
    dom = BeautifulSoup(response.content, "html.parser")
    element = dom.select(".prodcutInfo")[0]

    for i in range(1,len(element.select("dt"))+1) :
        
        if element.select_one("dt:nth-of-type({})".format(i)).text != "금리" :
            info.append({
            "product_idx" : idx,
            "title_txt" : element.select_one("dt:nth-of-type({})".format(i)).text,
            "desc_txt" : element.select_one("dd:nth-of-type({})".format(i)).text\
                .replace("\n","").replace("\r","").replace("\t","")
            })
        else :         
            driver = webdriver.Chrome()
            url = "https://www.kebhana.com" + element.select_one("dd:nth-of-type({}) > .tbBox > a".format(i)).get("href")
            driver.get(url)
            time.sleep(3)
            info.append({
            "product_idx" : idx,
            "title_txt" : element.select_one("dt:nth-of-type({})".format(i)).text,
            "desc_txt" : driver.find_element_by_css_selector("#hanaContentDiv > div > table > tbody").text.split("\n")[-1].split(" ")[-1]
            })
            driver.quit()

    return info

In [11]:
import re

In [12]:
for idx, data in items_df.iterrows():
    detail_datas(data.link, idx)

In [13]:
item_details_df = pd.DataFrame(info)
item_details_df[:20]

,product_idx,title_txt,desc_txt
0,0,상품특징,(스마트폰 전용 예금) 마케팅동의와 하나오픈뱅킹 서비스를 통하여 상품 신규 시 우대...
1,0,가입대상,실명의 개인 및 개인사업자
2,0,예금종류,정기예금
3,0,가입기간,1년
4,0,가입금액,"1백만원 이상 ~ 3천만원 이내 (단, 오픈뱅킹 1일 이체한도는 1천만원)"
5,0,이자지급방법,만기일시지급식
6,0,상품특징,(스마트폰 전용 예금) 마케팅동의와 하나오픈뱅킹 서비스를 통하여 상품 신규 시 우대...
7,0,가입대상,실명의 개인 및 개인사업자
8,0,예금종류,정기예금
9,0,가입기간,1년


In [264]:
item_details_df[item_details_df["title_txt"] == "금리"]

,product_idx,title_txt,desc_txt
6,0,금리,0.8%
20,1,금리,0.4%
34,2,금리,0.75%
48,3,금리,0.75%
60,4,금리,0.5%
73,5,금리,0.65%
85,6,금리,1.0%
98,7,금리,0.65%
110,8,금리,0.6%
122,9,금리,0.85%
